## Tensor traversal

Illustrate both concordant and discordant traversal of the ranks of a rank-2 tensor.

First, include some libraries


In [ ]:
%run ../prelude.py --style=tree --animation=movie

## Initialize setup

The following cell just creates some sliders to control the creation of the input operand tensors. Interesting variants to try:

- Density = 0.2
- Density = 1.0

In [ ]:
# Initial values

M = 8
K = 6
density = 0.4
seed = 10


tm = TensorMaker()
tm.addTensor("A",
             rank_ids=["M", "K"],
             shape=[M, K],
             density=density,
             color="blue",
             seed=seed)
tm.displayControls()




## Create an input tensors

Because this notebook tends to use both the original and rank swapped versions of the operands, the tensor names are suffixed with the ordered names of the ranks.


In [ ]:
a_MK = tm.makeTensor("A")
a_MK.setName("a_MK")
displayTensor(a_MK)


## Concordant traversal of the entire tensor

Concordant traversal of the entire tensor, i.e., the top rank coordinate (__m__) moving slowest, just requires nested loops that interate through the fibers in the tensor.

In [ ]:
a_m = a_MK.getRoot()

canvas = createCanvas(a_MK)
print(f"(M, K) -> Value")

for m, a_k in a_m:
    for k, a_val in a_k:
        print(f"({m}, {k}) -> {a_val}")
        canvas.addFrame((m,k))
        
displayCanvas(canvas)
        

## Discordant traversal of the entire tensor

If we want to traverse the tensor in a reverse rank order, i.e., the lower rank coordinate (__k__) moving slowest, we cannot iterate through the fibers in the desired order, so we must iterate over the coordinates, and use __Fiber.getPayload()__ to get the values in the desired order.

Note: using the keyword "allocate=False" causes __Fiber.getPayload()__ to return __None__ for empty coordinates.

In [ ]:
a_m = a_MK.getRoot()

canvas = createCanvas(a_MK)
print(f"(K, M) -> Value")

for k in range(K):
    for m in range(M):
        a_val = a_MK.getPayload(m, k, allocate=False)
        if a_val is not None:
            print(f"({k}, {m}) -> {a_val}")
            canvas.addFrame((m, k))
        
displayCanvas(canvas)
        

## Concordant traversal of a given tile in a tensor

Given an _axis aligned hyper-rectangle_ (AAHR) specified as a start coordinate and size (in coordinates) for each rank, we can traverse the tile using __Fiber.getRange()__.

In [ ]:
# sample delta AAHR specified as a start coord and size

m_range = [0,3]
k_range = [3,3]

a_m = a_MK.getRoot()

canvas = createCanvas(a_MK)
print(f"(M, K) -> Value")

for m, a_k in a_m.getRange(*m_range):
    for k, a_val in a_k.getRange(*k_range):
        print(f"({m}, {k}) -> {a_val}")
        canvas.addFrame((m,k))
        
displayCanvas(canvas)
        

## Disconcordant traversal of a given tile in a tensor

Discordant traversal of a AAHR in a tensor can be accomplished using __Fiber.getPayload()__.

In [ ]:
# sample delta AAHR specified as a start coord and size

m_range = [0,3]
k_range = [3,3]

a_m = a_MK.getRoot()

canvas = createCanvas(a_MK)
print(f"(K, M) -> Value")

for k in range(k_range[0], k_range[0]+k_range[1]):
    for m in range(m_range[0],m_range[0]+m_range[1]):
        a_val = a_MK.getPayload(m, k, allocate=False)
        if a_val is not None:
            print(f"({k}, {m}) -> {a_val}")
            canvas.addFrame((m, k))
        
displayCanvas(canvas)
        

## Testing Area